In [122]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import wget
import zipfile
import warnings
import patoolib
import shutil
import os
import glob
import matplotlib.pyplot as plt

# Rotina para Download e leitura dos arquivos

## Dowloading the .ZIP and .RAR files

In [123]:
warnings.filterwarnings('ignore')

# Request
r1 = requests.get('https://www.gov.br/prf/pt-br/acesso-a-informacao/dados-abertos/dados-abertos-acidentes')
r1.status_code
 
# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html.parser')

# useful identification
coverpage_useful = soup1.find_all('a', class_='external-link')

coverpage_useful = np.delete(coverpage_useful, 6)

# links to dowload

links = np.array([])


for i in range(len(coverpage_useful[4:20])):
    links = np.append(links, coverpage_useful[4:20][i]['href'])



In [124]:
url = links
PATH = str('C:/Users/Igor/Desktop/TCC/')

if os.path.isfile(str('C:/Users/Igor/Desktop/TCC/zips')) == False:
    shutil.rmtree(str('C:/Users/Igor/Desktop/TCC/zips'))
    os.makedirs(str('C:/Users/Igor/Desktop/TCC/zips'))

else:
    os.makedirs(str('C:/Users/Igor/Desktop/TCC/zips'))

for i in range(len(url)):

    if i <= 5:
        wget.download(url[i], PATH + str('zips/') + str(coverpage_useful[4:20][i].get_text() + str('.zip')))

    else:
        wget.download(url[i], PATH + str('zips/') + str(coverpage_useful[4:20][i].get_text() + str('.rar')))


## Extracting .ZIP and .RAR Archives & reading them

In [125]:
warnings.filterwarnings('ignore')

if os.path.isfile(str('C:/Users/Igor/Desktop/TCC/scripts/dados')) == False:
    shutil.rmtree(str('C:/Users/Igor/Desktop/TCC/scripts/dados'))
    os.makedirs(str('C:/Users/Igor/Desktop/TCC/scripts/dados'))

else:
    os.makedirs(str('C:/Users/Igor/Desktop/TCC/scripts/dados'))


for i in range(len(url)):

    if i <= 5:
        patoolib.extract_archive(PATH + str('zips/') + coverpage_useful[4:20][i].get_text() + str('.zip'), outdir='dados')

    else:
        patoolib.extract_archive(PATH + str('zips/') + coverpage_useful[4:20][i].get_text() + str('.rar'), outdir='dados')


path_df = np.array([])
for i in glob.glob('C:/Users/Igor/Desktop/TCC/scripts/dados/*'):
    path_df = np.append(path_df, i)


for i in range(len(path_df)):
    if i == 0:
        df = pd.read_csv(path_df[i], sep = ';', encoding='latin')
        
    else:
        df = df.append(
            pd.read_csv(path_df[i], sep = ';', encoding='latin'), ignore_index = True
            )


patool: Extracting C:/Users/Igor/Desktop/TCC/zips/2022 (até Setembro).zip ...
patool: ... C:/Users/Igor/Desktop/TCC/zips/2022 (até Setembro).zip extracted to `dados'.
patool: Extracting C:/Users/Igor/Desktop/TCC/zips/2021.zip ...
patool: ... C:/Users/Igor/Desktop/TCC/zips/2021.zip extracted to `dados'.
patool: Extracting C:/Users/Igor/Desktop/TCC/zips/2020.zip ...
patool: ... C:/Users/Igor/Desktop/TCC/zips/2020.zip extracted to `dados'.
patool: Extracting C:/Users/Igor/Desktop/TCC/zips/2019.zip ...
patool: ... C:/Users/Igor/Desktop/TCC/zips/2019.zip extracted to `dados'.
patool: Extracting C:/Users/Igor/Desktop/TCC/zips/2018.zip ...
patool: ... C:/Users/Igor/Desktop/TCC/zips/2018.zip extracted to `dados'.
patool: Extracting C:/Users/Igor/Desktop/TCC/zips/2017.zip ...
patool: ... C:/Users/Igor/Desktop/TCC/zips/2017.zip extracted to `dados'.
patool: Extracting C:/Users/Igor/Desktop/TCC/zips/2016.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- C:\Users\Igor\Desktop\TCC\zips

# EDA

In [126]:
print('Início do período:', pd.to_datetime(df['data_inversa']).min())
print('Final do período:', pd.to_datetime(df['data_inversa']).max())

Início do período: 2007-01-01 00:00:00
Final do período: 2022-09-30 00:00:00


In [127]:
df.head(3)

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,...,latitude,longitude,regional,delegacia,uop,ano,pessoas,ignorados,feridos,veiculos
0,17.0,15.0,2017-01-01,domingo,01:45:00,RS,116.0,"34,9",VACARIA,Defeito Mecânico no Veículo,...,"-28,5071196","-50,941176",SPRF-RS,DEL05-RS,UOP03-DEL05-RS,NaN,NaN,NaN,NaN,NaN
1,20.0,1541.0,2017-01-01,domingo,01:00:00,PR,376.0,636,TIJUCAS DO SUL,Velocidade Incompatível,...,"-25,754","-49,1266",SPRF-PR,DEL01-PR,DEL7/1-UOP08/PR,NaN,NaN,NaN,NaN,NaN
2,69.0,11707.0,2017-01-01,domingo,04:40:00,BA,101.0,65,ENTRE RIOS,Condutor Dormindo,...,"-11,9618","-38,0953",SPRF-BA,DEL01-BA,UOP04-DEL01-BA,NaN,NaN,NaN,NaN,NaN


In [128]:
df.columns

Index(['id', 'pesid', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br',
       'km', 'municipio', 'causa_acidente', 'tipo_acidente',
       'classificacao_acidente', 'fase_dia', 'sentido_via',
       'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
       'id_veiculo', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo',
       'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos',
       'feridos_leves', 'feridos_graves', 'mortos', 'latitude', 'longitude',
       'regional', 'delegacia', 'uop', 'ano', 'pessoas', 'ignorados',
       'feridos', 'veiculos'],
      dtype='object')

In [129]:
#Número de BR's

len(np.unique([i for i in df['br'].dropna() if i != np.nan]))

326

In [130]:
#Número de municipio

len(np.unique(df['municipio'].dropna()))

2330

In [131]:
np.unique(df['condicao_metereologica'].dropna())

array(['(null)', 'Ceu Claro', 'Chuva', 'Céu Claro', 'Garoa/Chuvisco',
       'Granizo', 'Ignorada', 'Ignorado', 'Neve', 'Nevoeiro/Neblina',
       'Nevoeiro/neblina', 'Nublado', 'Sol', 'Vento'], dtype=object)